In [1]:
# import argparse
# from trl import SFTTrainer
# from transformers import TrainingArguments, DataCollatorForSeq2Seq
# from unsloth import is_bfloat16_supported
# from finetune.lib import prepare_peft_model_n_tokenizer, \
#         prepare_dataset, show_memory_stat, max_seq_length, \
#         seed_everything 
# from unsloth import FastLanguageModel
# from unsloth.chat_templates import train_on_responses_only
# import pdb 

In [2]:
# dataset_path = "/home/vaibhav/LLMs-TimeSeries/software/Output_dataset_text_test.json" 

# class args1:
#     def __init__(self, model, dataset_dir, bs, epoch, max_round, max_steps):
#         self.model = model
#         self.dataset_dir = dataset_dir
#         self.bs = bs
#         self.epoch = epoch
#         self.max_round = max_round
#         self.max_steps = max_steps

        
# # Creating an instance of the Person class
# args = args1("unsloth/Llama-3.2-1B-Instruct", dataset_path, 2, 4, 1, -1)

In [3]:
# seed = 3407
# seed_everything(seed)
# model, tokenizer = prepare_peft_model_n_tokenizer(
#     model_name=args.model
# )
# # train_dataset, eval_dataset  = prepare_dataset(tokenizer=tokenizer)
# train_dataset, eval_dataset  = prepare_dataset(tokenizer=tokenizer, dataset_dir=args.dataset_dir)

# # if args.max_steps != -1:
# #     args.epoch = 1
# print(train_dataset)
# print(train_dataset[0])

In [4]:
# trainer_args = TrainingArguments(
#         per_device_train_batch_size = args.bs,
#         gradient_accumulation_steps = 4,
#         warmup_steps = 5,
#         num_train_epochs = args.epoch, # Set this for 1 full training run.
#         # max_steps = args.max_steps,
#         learning_rate = 2e-4,
#         fp16 = not is_bfloat16_supported(),
#         bf16 = is_bfloat16_supported(),
#         logging_steps = 1,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = seed,
#         output_dir = "outputs",
#         report_to = "wandb", # Use this for WandB etc
#         # add evaluation arguments
#         fp16_full_eval = False,
#         per_device_eval_batch_size = 2,
#         eval_accumulation_steps = 4,
#         eval_strategy = "epoch",
#         eval_steps = 1,
#     )

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = train_dataset,
#     eval_dataset = eval_dataset,
#     dataset_text_field = "text",
#     max_seq_length = max_seq_length,
#     data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
#     dataset_num_proc = 2,
#     packing = False, # Can make training 5x faster for short sequences.
#     args = trainer_args,
# )

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "/home/vaibhav/LLMs-TimeSeries/software/Archive/Llama-3.2-1B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/vaibhav/miniconda3/envs/vai_llama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [6]:
import json

def read_json(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
    return data 


dataset_dir = "/home/vaibhav/LLMs-TimeSeries/software/Output_dataset_text_test.json" 

dataset = read_json(dataset_dir)
print(dataset[0]['user'])
print(dataset[0]['assistant'])


Mentioned data is from Air Quality Chemical Multisensory Device which uses tungsten oxide as its sensing material and is designed to detect nitrogen dioxide (NO2) gas. It is deployed in a significantly polluted area, at road level, within an Italian city.  Average value on those four nights is [1066.8, 1029.8, 953.9, 1031.0, 948.0, 919.9, 1085.0, 1169.1, 1147.0, 1102.0, 958.0, 946.2], [946.8, 952.1, 939.2, 996.9, 1017.2, 936.2, 846.9, 914.9, 917.2, 943.9, 870.2, 898.1], [834.0, 843.1, 875.8, 831.8, 825.1, 788.0, 819.0, 875.2, 892.1, 804.9, 782.1, 781.8] and [777.1, 760.0, 748.9, 737.9, 726.1, 752.9, 989.1, 1025.0, 1017.1, 906.1, 842.2, 831.0]. Estimate the expected average Nitrogen Dioxide value for the subsequent night.
791


In [11]:
count = 0
ground_truth = []
output_result = []
image_type = "spectrogram"

for i in range(len(dataset)):
        
    # image = dataset[i]["image"]
    # en_query = dataset[i]["query"]
    ground_truth.append( int(dataset[i]["assistant"]) )
    # print(en_query)

    prompt = dataset[i]['user']

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")  # Assuming you have a CUDA device
    with torch.no_grad():
        output_ids = model.generate(input_ids, num_beams=4, max_length=1024)
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(int(generated_text[-4:]))
    output_result.append( int(generated_text[-4:]) )



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1002
1322


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1002
1578


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

1182


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1290
1322


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1322
1322


In [12]:
print(ground_truth)
print(output_result)
print(len(ground_truth) == len(dataset))

[791, 1618, 1106, 1768, 1263, 1230, 2011, 1711, 1325, 1683, 1892, 1908, 1664, 1829, 1490, 1436, 1460, 1601, 1423, 1171, 1550, 1212, 1039, 895, 1294, 1778, 1182, 1290, 1261, 1253, 1095]
[1002, 1322, 1002, 1578, 1322, 1579, 1872, 1872, 1832, 1839, 1872, 1539, 1762, 1872, 1322, 1459, 1322, 1572, 1342, 1342, 1829, 1172, 1022, 1022, 1532, 1832, 1182, 1290, 1322, 1322, 1322]
True


In [13]:
import math

mae_error = 0
rmse_error = 0
count = 0

for count in range(len(output_result)):
    mae_error = mae_error + abs(ground_truth[count] - output_result[count] )
    rmse_error = rmse_error + pow( (ground_truth[count] - output_result[count] ) , 2)
    count = count + 1

mae_error = mae_error / len(dataset)
rmse_error = math.sqrt(rmse_error / len(dataset))
# if count == len(dataset):
#     mae_error = mae_error / len(dataset)
# else:
#     print("error at count: " + str(count))
#     print(response)
print(mae_error)
print(rmse_error)



142.70967741935485
186.55155821104188
